# Filtering by custodial sentence length

In [1]:
import pandas as pd

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from src.data.processing import filter_sentence_length

In [24]:
df = filter_sentence_length.load_and_process_data()

2025-06-10 12:05:14,348 - INFO - Loaded data from data/interim/women_cust_comm_sus.csv
2025-06-10 12:05:14,649 - INFO - Filtering data for custodial sentences...
2025-06-10 12:05:14,660 - INFO - Grouping sentence lengths...
2025-06-10 12:05:14,749 - INFO - Sentence lengths grouped and recategorised.
2025-06-10 12:05:14,750 - INFO - Grouping data by PFA, year, and sentence length...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 82823 entries, 0 to 331014
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   year          82823 non-null  int64   
 1   pfa           82823 non-null  category
 2   sex           82823 non-null  category
 3   age_group     82823 non-null  category
 4   offence       82823 non-null  category
 5   outcome       82823 non-null  category
 6   sentence_len  82823 non-null  category
 7   freq          82823 non-null  int64   
dtypes: category(6), int64(2)
memory usage: 2.4 MB


In [6]:
categories = df['sentence_len'].cat.categories
categories

Index(['Less than 6 months', '6–12 months', 'Over 12 months'], dtype='object')

In [25]:
df

,pfa,year,sentence_len,freq
0,Avon and Somerset,2010,Less than 6 months,113
1,Avon and Somerset,2010,6–12 months,16
2,Avon and Somerset,2010,Over 12 months,49
3,Avon and Somerset,2011,Less than 6 months,142
4,Avon and Somerset,2011,6–12 months,21
...,...,...,...,...
1839,Wiltshire,2023,6–12 months,4
1840,Wiltshire,2023,Over 12 months,10
1841,Wiltshire,2024,Less than 6 months,22
1842,Wiltshire,2024,6–12 months,1
